In [1]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer 

In [2]:
import pandas as pd 

In [3]:
model1 = "rajj0/abstract_ai" 

tokenizer = AutoTokenizer.from_pretrained("rajj0/abstract_ai", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left" 

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
model1 = AutoModelForCausalLM.from_pretrained(model1) 

In [ ]:
# temperatures = [0.8, 0.7, 0.5, 0.6]
# for temperature in temperatures: 
#     input_ids = tokenizer.encode(prompt, return_tensors="pt")
#     output = model1.generate(input_ids, do_sample=True, max_length=1000, temperature=temperature, top_p=0.8)

#     generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
#     prompt_length = len(prompt)
#     response_start_idx = generated_text.find(prompt) + prompt_length
#     response = generated_text[response_start_idx:].strip() 
#     print(f"Temperature: {temperature}\nResponse: {response}\n")
# # 

In [ ]:
prompt = f"[INST] Generate abstract for the key points\n1. Chimera Design: The design of Chimera is broken into a core and extensions. The core provides basic services and visualization, while the extensions are responsible for higher level functionality, allowing third-party developers to incorporate new features according to their needs.\n2. Multiscale Extension: The Multiscale extension of Chimera allows users to visualize large-scale molecular assemblies such as viral coats. By providing a scale-based approach, it enhances the understanding of molecular structures and interactions in biological research.\n3. Collaboratory Extension: Offering the ability for researchers based in different locations to share a Chimera session interactively, the Collaboratory extension significantly improves collaboration capacity. Through this shared environment, researchers can conduct simultaneous examinations and share insights in real-time.\n4. Other Extensions: Other extensions such as Multalign Viewer, ViewDock, Movie, and Volume Viewer offer a diverse set of features. They allow the display of multiple sequence alignments, screening of docked ligand orientations, replay of molecular dynamics trajectories, and analysis of volumetric data respectively.\n5. Real-World Usage of Chimera: The abstract also discusses the practical usage of Chimera in real-world situations, pointing out its wide applicability and impact in the field of molecular biology and bioinformatics \n . [/INST]"


In [ ]:
# # prompt = f"[INST] Generate abstract for the key points\n1. Chimera Design: The design of Chimera is broken into a core and extensions. The core provides basic services and visualization, while the extensions are responsible for higher level functionality, allowing third-party developers to incorporate new features according to their needs.\n2. Multiscale Extension: The Multiscale extension of Chimera allows users to visualize large-scale molecular assemblies such as viral coats. By providing a scale-based approach, it enhances the understanding of molecular structures and interactions in biological research.\n3. Collaboratory Extension: Offering the ability for researchers based in different locations to share a Chimera session interactively, the Collaboratory extension significantly improves collaboration capacity. Through this shared environment, researchers can conduct simultaneous examinations and share insights in real-time.\n4. Other Extensions: Other extensions such as Multalign Viewer, ViewDock, Movie, and Volume Viewer offer a diverse set of features. They allow the display of multiple sequence alignments, screening of docked ligand orientations, replay of molecular dynamics trajectories, and analysis of volumetric data respectively.\n5. Real-World Usage of Chimera: The abstract also discusses the practical usage of Chimera in real-world situations, pointing out its wide applicability and impact in the field of molecular biology and bioinformatics \n . [/INST]"
# pipe = pipeline(task="text-generation", model=model1, tokenizer=tokenizer, max_length=1000, temperature=0.7, top_p=0.92)
# # result = pipe(prompt)
# print(result)

The abstract discusses the design of Chimera, a molecular visualization tool, which is divided into a core and extensions. The core provides basic services and visualization, while the extensions offer higher-level functionality, allowing third-party developers to incorporate new features. The Multiscale extension of Chimera allows users to visualize large-scale molecular assemblies, such as viral coats, at multiple scales. The Collaboratory extension enables researchers in different locations to share a Chimera session interactively, enhancing collaboration capacity. Other extensions, such as Multalign Viewer, ViewDock, Movie, and Volume Viewer, offer a range of features, including the display of multiple sequence alignments, screening of docked ligand orientations, replay of molecular dynamics trajectories, and analysis of volumetric data. The abstract also highlights the practical usage of Chimera in real-world situations, demonstrating its wide applicability and impact in the field of molecular biology and bioinformatics.

In [ ]:
# import pandas as pd

# df = pd.read_csv("prompt_train.csv")

# r = []
# for index, row in df.iterrows():
#     prompt_text = row["prompt"]
#     output_prompt = f"[INST] {prompt_text} [/INST]"
#     temp =[0.7, 0.9]
   
#     for i in temp:
#         pipe = pipeline(task="text-generation", model=model1, tokenizer=tokenizer, max_length=1000, temperature=i, top_p=0.92)
#         result = pipe(output_prompt)
#         r.append(result)
#     print(result)
    


In [ ]:
# r

In [10]:
pipe = pipeline(
                    task="text-generation", 
                    model=model1, 
                    tokenizer=tokenizer, 
                    max_length=512, 
                    temperature=0.7, 
                    top_p=0.92,
                    do_sample=True,
                    device="cuda:0"
                )
result = pipe(prompt, truncate=True)
result

OutOfMemoryError: CUDA out of memory. Tried to allocate 172.00 MiB (GPU 0; 15.77 GiB total capacity; 15.39 GiB already allocated; 24.38 MiB free; 15.39 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [37]:
df = pd.read_csv("prompt_train.csv")

prompts = []
responses_A = []
responses_B = []

for index, row in df.iterrows():
    prompt_text = row["prompt"]
    output_prompt = prompt_text
    temp = [0.7, 1]

    generated_responses_A = []
    generated_responses_B = []
    
    for i in temp:
        pipe = pipeline(task="text-generation", model=model1, tokenizer=tokenizer, max_length=1000, temperature=i, top_p=0.92)
        result = pipe(output_prompt)
        if i == 0.7:
            generated_responses_A.append(result[0]['generated_text'])
        else:
            generated_responses_B.append(result[0]['generated_text'])
    
    prompts.append(prompt_text)
    responses_A.append(generated_responses_A)
    responses_B.append(generated_responses_B)

formatted_responses = []
for i in range(len(prompts)):
    formatted_response = f"### prompt: {prompts[i]}\n"
    formatted_response += f"### responseA: {responses_A[i][0]}\n"
    formatted_response += f"### responseB: {responses_B[i][0]}\n"
    formatted_responses.append(formatted_response)


output_df = pd.DataFrame({"formatted_response": formatted_responses})

output_df.to_csv("formatted_responses.csv", index=False)


/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:363: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:368: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.92` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


KeyboardInterrupt: 